이 자료는 Hugging Face Pipeline API를 이용한 챗봇 튜토리얼로 작성되었습니다.  
2025년 9월 14일에 정상 동작을 확인하였습니다.  

1. Hugging Face 회원가입(Sign Up)

    - [Hugging Face 바로가기: https://huggingface.co/](https://huggingface.co/)  
    - Sign Up 따라하기  

2. Hugging Face의 Access Token 생성  

    - 우측상단 (三) 메뉴 -> Settings -> Tokens -> +Create new token
    - 읽기 전용(Read) / 쓰기(Write) / 관리자(Admin) 중 목적에 맞게 발급  
    - 발급된 코드는 다시 볼 수 없기 때문에 별도로 보관  

In [ ]:
from huggingface_hub import login
# login(token="hf_xxx_your_token")

3. Transformer 설치  

In [ ]:
!pip install --upgrade transformers

4. Colab에서 Hugging Face의 Access Token을 이용하여 로그인(LogIn)  

In [ ]:
!hf auth login --token token --add-to-git-credential

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_http.py", line 409, in hf_raise_for_status
    response.raise_for_status()
  File "/usr/local/lib/python3.12/dist-packages/requests/models.py", line 1026, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPError: 401 Client Error: Unauthorized for url: https://huggingface.co/api/whoami-v2

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/huggingface_hub/hf_api.py", line 1782, in whoami
    hf_raise_for_status(r)
  File "/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_http.py", line 482, in hf_raise_for_status
    raise _format(HfHubHTTPError, str(e), response) from e
huggingface_hub.errors.HfHubHTTPError: 401 Client Error: Unauthorized for url: https://huggingface.co/api/whoami-v2 (Request ID: Root=1-68c6a538-44c846d85452

5. Git credential 설정 하기  

In [ ]:
!git config --global credential.helper store

6. Hugging Face 로그인 상태 확인  

In [ ]:
!hf auth whoami

gislee


7. 질의응답(Q&A) 모델 불러오기 (bert-large-uncased-whole-word-masking-finetuned-squad)

In [ ]:
from transformers import pipeline

qa = pipeline("question-answering")

context = """AI agents are intelligent systems that can perceive the environment,
make decisions, and take actions autonomously. They are increasingly powered by
large language models (LLMs) such as GPT or LLaMA."""

question = "What powers modern AI agents?"
result = qa(question=question, context=context)
print("질문:", question)
print("답변:", result['answer'])

No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 564e9b5 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

...

Fetching 0 files: 0it [00:00, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 0 files: 0it [00:00, ?it/s]

Device set to use cuda:0


질문: What powers modern AI agents?
답변: large language models


8. Hugging Face Pipeline API를 이용한 챗봇

In [1]:
# 1) Colab.로컬 공통: 설치
!pip -q install "transformers>=4.44.0" sentencepiece gradio

In [ ]:
# 2) 간단 챗봇 (Pipeline + 대화 이력 유지)
import torch
from transformers import pipeline, AutoTokenizer

MODEL_NAME = "Qwen/Qwen2.5-1.5B-Instruct"
device = 0 if torch.cuda.is_available() else -1

tok = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
chatbot = pipeline(
    "text-generation",
    model=MODEL_NAME,
    tokenizer=tok,
    device=device
)

SYSTEM_PROMPT = (
    "You are a helpful, concise assistant for a university AI Agent course. "
    "Answer in the user's language (Korean or English)."
)

def format_history(history, user_text, system_prompt=SYSTEM_PROMPT):
    """
    history: list[tuple[str,str]] = [(user, assistant), ...]
    user_text: 현재 입력
    """
    lines = [f"System: {system_prompt}"]
    for u, a in history:
        lines.append(f"User: {u}")
        lines.append(f"Assistant: {a}")
    lines.append(f"User: {user_text}")
    lines.append("Assistant:")
    return "\n".join(lines)

def generate_reply(history, user_text):
    prompt = format_history(history, user_text)
    out = chatbot(
        prompt,
        max_new_tokens=256,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.05,
        pad_token_id=tok.eos_token_id
    )[0]["generated_text"]
    # 간단 파싱: 마지막 "Assistant:" 뒤 텍스트를 응답으로 사용
    reply = out.split("Assistant:")[-1].strip()
    # 길이 과도 시 컷
    return reply[:1200]

# 🔁 터미널 대화 루프 (원하면 실행)
history = []
print("간단 챗봇 시작! 종료: exit/quit")
while True:
    user = input("You: ").strip()
    if user.lower() in {"exit","quit"}:
        print("Bye!")
        break
    bot = generate_reply(history, user)
    history.append((user, bot))
    print("Bot:", bot)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Device set to use cuda:0


간단 챗봇 시작! 종료: exit/quit
You: What are the AI Agents?
Bot: In the context of computer science and artificial intelligence, an AI agent is a software program that can interact with its environment and learn from experiences to achieve goals. These agents can be autonomous or controlled by humans, and they are designed to perform tasks that typically require human-like intelligence, such as decision-making, learning, problem-solving, and perception.

AI agents can be categorized into various types based on their functionality, including:

1. Reinforcement Learning Agents: These agents learn through trial and error by interacting with an environment and receiving rewards or penalties for their actions.

2. Supervised Learning Agents: These agents learn by being exposed to labeled data, where the input data is paired with the correct output.

3. Unsupervised Learning Agents: These agents learn without being given any labeled data; they discover patterns and structure within the data themsel

In [ ]:
# 3) (선택) Gradio 웹 UI
import gradio as gr

history = []  # [(user, bot), ...]

def respond(msg, chat_hist):
    global history
    # gradio chat_hist: [[user, bot], ...]
    history = [(u or "", b or "") for (u, b) in (chat_hist or [])]
    bot = generate_reply(history, msg)
    history.append((msg, bot))
    chat_hist = (chat_hist or []) + [[msg, bot]]
    return "", chat_hist

def clear():
    global history
    history = []
    return []

with gr.Blocks() as demo:
    gr.Markdown("## 🤗 Hugging Face pipeline 기반 간단 챗봇")
    gr.Markdown(f"- 모델: **{MODEL_NAME}**  |  한국어/영어 자동 응답")
    chat = gr.Chatbot(height=380)
    txt = gr.Textbox(placeholder="질문을 입력하세요…", autofocus=True)
    btn = gr.Button("대화 리셋")

    txt.submit(respond, [txt, chat], [txt, chat])
    btn.click(clear, None, chat)

demo.launch(debug=False)